# Kapitel 8 – Fehlerbehandlung und robuste Programme

Dieses Notebook begleitet das Kapitel **„Fehlerbehandlung und robuste Programme“**.

Ziele:
- Fehlertypen verstehen und lesen lernen
- `try`, `except`, `else`, `finally` gezielt einsetzen
- mit `raise` Eingaben validieren
- einfache Formen von Logging nutzen
- eine kleine, robuste Auswertung historischer Daten bauen

Die Beispiele hier sind **neu** und unterscheiden sich von den Beispielen im Skript,
aber illustrieren dieselben Prinzipien.

## 1. Häufige Fehlertypen im historischen Kontext

Wir erzeugen absichtlich ein paar typische Fehler, um sie zu erkennen und zu verstehen.


In [ ]:
# NameError: Variable nicht definiert
try:
    print(year_of_plague)
except NameError as e:
    print("NameError erkannt:", e)

In [ ]:
# TypeError: ungeeignete Operation für den Datentyp
try:
    victims = "12000"  # String statt Zahl
    print(victims / 2)
except TypeError as e:
    print("TypeError erkannt:", e)

In [ ]:
# IndexError: Zugriff ausserhalb der Liste
years = [1347, 1348, 1349]
try:
    print(years[3])
except IndexError as e:
    print("IndexError erkannt:", e)

## 2. `try` / `except`: Fehler abfangen statt Absturz

Beispiel: Wir haben Angaben zur Bevölkerung und Todesfällen in einer Stadt während einer Epidemie.
In einigen Fällen fehlen Daten (`None`). Wir wollen robust berechnen, nicht abstürzen.

In [ ]:
data = [
    {"city": "Firenze", "population": 90000, "deaths": 30000},
    {"city": "Siena", "population": 50000, "deaths": None},   # unvollständige Daten
    {"city": "Lucca", "population": 30000, "deaths": 8000}
]

for entry in data:
    city = entry["city"]
    try:
        rate = entry["deaths"] / entry["population"]
        print(f"{city}: Sterberate {rate:.2%}")
    except TypeError:
        print(f"{city}: Daten unvollständig – keine Sterberate berechenbar.")

## 3. `else`: Nur ausführen, wenn kein Fehler auftritt

`else` wird nur dann ausgeführt, wenn der `try`-Block **ohne** Exception durchläuft.

In [ ]:
text = "Anno Domini 1521 Martinus Lutherus Wormatiae adfuit."

try:
    words = text.split()
    year = int(words[2])  # hier bewusst auf Position 2
except (IndexError, ValueError) as e:
    print("Fehler beim Extrahieren des Jahres:", e)
else:
    print("Extrahiertes Jahr:", year)

## 4. `finally`: Aufräumen unabhängig vom Ergebnis

Angenommen, wir lesen (simuliert) eine Datei mit Ratsprotokollen.
Unabhängig davon, ob das Lesen klappt, soll eine Abschlussmeldung erfolgen.

In [ ]:
filename = "ratsprotokoll_1620.txt"  # existiert hier nicht wirklich

try:
    f = open(filename, encoding="utf-8")
    content = f.read()
    print(content[:80])
except FileNotFoundError as e:
    print("Datei nicht gefunden:", e)
finally:
    print("Dateizugriff versucht (erfolgreich oder nicht).")

## 5. `raise`: Eingaben validieren (historische Daten)

Wir implementieren einfache fachlogische Regeln:
- Jahr `0` ist historisch (im üblichen v. Chr. / n. Chr.-Schema) ungültig.
- Das Geburtsjahr darf nicht in der Zukunft liegen.

Verstösse werden explizit mit `raise` gemeldet.

In [ ]:
from datetime import datetime

CURRENT_YEAR = datetime.now().year

def validate_birth_year(year: int) -> None:
    """Validiert ein (vereinfachtes) historisches Geburtsjahr."""
    if year == 0:
        raise ValueError("Jahr 0 ist historisch nicht definiert.")
    if year > CURRENT_YEAR:
        raise ValueError("Geburtsjahr liegt in der Zukunft.")

def format_year(year: int) -> str:
    """Gibt eine einfache v. Chr. / n. Chr.-Darstellung zurück."""
    validate_birth_year(year)
    return f"{abs(year)} v. Chr." if year < 0 else f"{year} n. Chr."

# Beispiele
for y in [-356, 800, 0, 3000]:
    try:
        print(y, "→", format_year(y))
    except ValueError as e:
        print(y, "→ Fehler:", e)

## 6. Spezifische vs. generische `except`-Blöcke

Wir demonstrieren, wie man **gezielt** auf verschiedene Fehler reagiert.
Beispiel: Zugriff auf ein kleines Personen-Dataset mit Geburtsjahren.

In [ ]:
people = {
    "Herodot": -484,
    "Thukydides": -460,
    "Anna Komnene": 1083
}

def print_person_year(name: str):
    try:
        year = people[name]
        print(f"{name}: {format_year(year)}")
    except KeyError:
        print(f"{name}: nicht in der Datenbasis.")
    except ValueError as e:
        print(f"{name}: fehlerhafte Jahresangabe – {e}")

print_person_year("Herodot")
print_person_year("Unbekannt")

## 7. Einfaches Logging statt nur `print`

Ein kurzer Einblick: Wir loggen Warnungen, wenn Daten unplausibel sind.
So wird unser Code nachvollziehbarer und robuster als mit verstreuten `print`-Befehlen.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

entries = [
    {"city": "Roma", "year": 64, "event": "Fire"},
    {"city": "Roma", "year": 8064, "event": "Typo"},  # offensichtlich falsch
]

for e in entries:
    year = e["year"]
    if year > CURRENT_YEAR:
        logging.warning(f"Unplausibles Jahr {year} für {e['city']} – Eintrag wird übersprungen.")
        continue
    logging.info(f"Verarbeite Ereignis {e['event']} in {e['city']} im Jahr {year}.")

## 8. Mini-Projekt: Robuste Auswertung eines historischen Datensatzes

Wir simulieren einen kleinen Datensatz mit Informationen zu Inschriften:

- `id`: Identifikator
- `year`: Jahr (kann fehlen oder fehlerhaft sein)
- `text`: Kurzbeschreibung

Ziel: robust zählen, wie viele Inschriften pro grober Periode vorliegen.
Wir verwenden `try`/`except` und `raise`, um den Code stabil zu halten.

In [ ]:
inscriptions = [
    {"id": 1, "year": -50, "text": "Dedication to a deity."},
    {"id": 2, "year": 120, "text": "Milestone inscription."},
    {"id": 3, "year": None, "text": "Fragment, undated."},
    {"id": 4, "year": 0, "text": "Corrupt data."},
    {"id": 5, "year": 450, "text": "Christian inscription."}
]

def classify_period(year: int) -> str:
    """Sehr grobe Periodisierung, nur als Beispiel."""
    validate_birth_year(year)  # nutzt unsere frühere Logik (kein Jahr 0, nicht in der Zukunft)
    if year < 0:
        return "Vorchristlich"
    if year <= 300:
        return "Frühe Kaiserzeit"
    return "Spätantike"

counts = {}

for ins in inscriptions:
    try:
        year = ins["year"]
        if year is None:
            raise ValueError("Kein Jahr angegeben.")
        period = classify_period(year)
    except ValueError as e:
        logging.warning(f"Inschrift {ins['id']}: {e} – wird übersprungen.")
        continue
    else:
        counts[period] = counts.get(period, 0) + 1

counts

👉 Dieses Mini-Beispiel zeigt:
- Fehler (fehlende oder ungültige Jahre) bringen das Notebook nicht zum Absturz.
- Problemfälle werden gemeldet (`logging.warning`).
- Gültige Daten werden sauber ausgewertet.

Damit sind die Kernideen von Kapitel 8 praktisch umgesetzt.